In [1]:
import polygon_generation as poly
import Knot as knot_lib

Could not import cythonised chelpers, using Python alternative. This will give the same result, but is slower.


In [2]:
import wget
url = "https://raw.githubusercontent.com/thomaseddy/stick-knot-gen/master/data/stick_number_upper_bounds.csv"
filename = wget.download(url)

100% [................................................................................] 2426 / 2426

In [3]:
import pandas as pd

In [4]:
csv_data = pd.read_csv(filename, header=0)
knot_table = dict(zip(list(csv_data["knot"]), list(csv_data[" stick_number_ub"])))
knot_table

{'0_1': 3,
 '3_1': 6,
 '4_1': 7,
 '5_1': 8,
 '5_2': 8,
 '6_1': 8,
 '6_2': 8,
 '6_3': 8,
 '7_1': 9,
 '7_2': 9,
 '7_3': 9,
 '7_4': 9,
 '7_5': 9,
 '7_6': 9,
 '7_7': 9,
 '8_1': 10,
 '8_2': 10,
 '8_3': 10,
 '8_4': 10,
 '8_5': 10,
 '8_6': 10,
 '8_7': 10,
 '8_8': 10,
 '8_9': 10,
 '8_10': 10,
 '8_11': 10,
 '8_12': 10,
 '8_13': 10,
 '8_14': 10,
 '8_15': 10,
 '8_16': 9,
 '8_17': 9,
 '8_18': 9,
 '8_19': 8,
 '8_20': 8,
 '8_21': 9,
 '9_1': 10,
 '9_2': 10,
 '9_3': 10,
 '9_4': 10,
 '9_5': 10,
 '9_6': 11,
 '9_7': 10,
 '9_8': 10,
 '9_9': 10,
 '9_10': 10,
 '9_11': 10,
 '9_12': 10,
 '9_13': 10,
 '9_14': 10,
 '9_15': 10,
 '9_16': 10,
 '9_17': 10,
 '9_18': 11,
 '9_19': 10,
 '9_20': 10,
 '9_21': 10,
 '9_22': 10,
 '9_23': 11,
 '9_24': 10,
 '9_25': 10,
 '9_26': 10,
 '9_27': 10,
 '9_28': 10,
 '9_29': 9,
 '9_30': 10,
 '9_31': 10,
 '9_32': 10,
 '9_33': 10,
 '9_34': 9,
 '9_35': 9,
 '9_36': 11,
 '9_37': 10,
 '9_38': 10,
 '9_39': 9,
 '9_40': 9,
 '9_41': 9,
 '9_42': 9,
 '9_43': 9,
 '9_44': 9,
 '9_45': 9,
 '9_46': 9,

In [ ]:
def generate_and_identify_knot(size):
    ID=[]
    empty_knot_id =0    
    while ID ==[]:        
        k = knot_lib.Knot(size)        
        empty_knot_id +=1        
    print("Took "+str(empty_knot_id)+" to generate this one")
    
    return ID

generate_and_identify_knot(13)